# 1 Data Preparation
## 1.1 Planning data structure 
- I need to build a 3d dataset for transformer model analysis, first dimension is time, second is window size which is 7, third dimension is all the other features. 
- using numpy & pandas for data preparation
- using pytorch for building transformer models

## 1.2 feature adjacent matrix

- First process the raw data of adjacency to a binary matrix for future modeling.

In [20]:
import pandas as pd
import numpy as np

add_adj = True
add_dvmt = False
add_popu = False
add_vict = True

# Read the Excel file for adjacency data
file_path = r'./data/2016-6-14-CA_Counties_Distance_Pnt.csv'
df_adj = pd.read_csv(file_path)
county_map = {'Alameda': 1, 'Alpine': 2, 'Amador': 3, 'Butte': 4, 'Calaveras': 5, 'Colusa': 6, 'Contra Costa': 7, 'Del Norte': 8, 'El Dorado': 9, 'Fresno': 10, 'Glenn': 11, 'Humboldt': 12, 'Imperial': 13, 'Inyo': 14, 'Kern': 15, 'Kings': 16, 'Lake': 17, 'Lassen': 18, 'Los Angeles': 19, 'Madera': 20, 'Marin': 21, 'Mariposa': 22, 'Mendocino': 23, 'Merced': 24, 'Modoc': 25, 'Mono': 26, 'Monterey': 27, 'Napa': 28, 'Nevada': 29, 'Orange': 30, 'Placer': 31, 'Plumas': 32, 'Riverside': 33, 'Sacramento': 34, 'San Benito': 35, 'San Bernardino': 36, 'San Diego': 37, 'San Francisco': 38, 'San Joaquin': 39, 'San Luis Obispo': 40, 'San Mateo': 41, 'Santa Barbara': 42, 'Santa Clara': 43, 'Santa Cruz': 44, 'Shasta': 45, 'Sierra': 46, 'Siskiyou': 47, 'Solano': 48, 'Sonoma': 49, 'Stanislaus': 50, 'Sutter': 51, 'Tehama': 52, 'Trinity': 53, 'Tulare': 54, 'Tuolumne': 55, 'Ventura': 56, 'Yolo': 57, 'Yuba': 58}
county_map = {k + ' County': v for k, v in county_map.items()}
df_adj.replace(county_map, inplace=True)
df_adj.dropna(axis=0, inplace=True)
df_adj.sort_values(by=['Input_Cnty'], inplace=True, ignore_index=True)

distance_matrix = df_adj.pivot(index='Input_Cnty', columns='Near_Cnty', values='DIST')

# Ensure all counties are present in both rows and columns
counties = pd.Index(distance_matrix.index).union(distance_matrix.columns)
distance_matrix = distance_matrix.reindex(index=counties, columns=counties)

# Fill NaN with 0 or any other value if there's no direct distance provided
distance_matrix = distance_matrix.fillna(0)
adjacency_df = distance_matrix.copy()
adjacency_df.head()

C:\Users\42514\AppData\Local\Temp\ipykernel_25968\1786228755.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_adj.replace(county_map, inplace=True)


,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0
1.0,0.000000,168.876572,105.811776,184.511251,107.596900,141.316986,28.152873,397.382602,138.284448,205.321402,...,106.325683,62.932805,127.546843,225.789677,285.001794,245.587904,140.087610,331.304440,95.800962,152.525028
2.0,168.876572,0.000000,68.754223,155.765615,61.410721,174.921299,157.424646,400.807841,48.953616,141.749208,...,212.216016,122.227166,135.335110,215.780026,293.116004,218.661534,51.071749,358.993112,144.173918,121.540713
3.0,105.811776,68.754223,0.000000,124.375122,25.077892,120.586514,90.237441,369.423165,32.936050,168.469011,...,145.468114,77.007582,82.984871,182.179396,256.703907,235.696891,67.304116,359.758990,80.486741,84.743252
4.0,184.511251,155.765615,124.375122,0.000000,148.866389,62.148687,156.811013,247.221885,113.123738,288.308615,...,134.999146,191.244527,56.964412,60.220376,137.431321,359.097499,184.854220,483.314862,89.948422,39.640824
5.0,107.596900,61.410721,25.077892,148.866389,0.000000,144.884028,98.201755,394.416927,45.876764,143.599814,...,163.933949,64.339038,107.845609,207.084718,281.771913,210.714780,44.256991,336.506385,102.508004,109.313816


In [21]:
distance_matrix

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0
1.0,0.000000,168.876572,105.811776,184.511251,107.596900,141.316986,28.152873,397.382602,138.284448,205.321402,...,106.325683,62.932805,127.546843,225.789677,285.001794,245.587904,140.087610,331.304440,95.800962,152.525028
2.0,168.876572,0.000000,68.754223,155.765615,61.410721,174.921299,157.424646,400.807841,48.953616,141.749208,...,212.216016,122.227166,135.335110,215.780026,293.116004,218.661534,51.071749,358.993112,144.173918,121.540713
3.0,105.811776,68.754223,0.000000,124.375122,25.077892,120.586514,90.237441,369.423165,32.936050,168.469011,...,145.468114,77.007582,82.984871,182.179396,256.703907,235.696891,67.304116,359.758990,80.486741,84.743252
4.0,184.511251,155.765615,124.375122,0.000000,148.866389,62.148687,156.811013,247.221885,113.123738,288.308615,...,134.999146,191.244527,56.964412,60.220376,137.431321,359.097499,184.854220,483.314862,89.948422,39.640824
5.0,107.596900,61.410721,25.077892,148.866389,0.000000,144.884028,98.201755,394.416927,45.876764,143.599814,...,163.933949,64.339038,107.845609,207.084718,281.771913,210.714780,44.256991,336.506385,102.508004,109.313816
6.0,141.316986,174.921299,120.586514,62.148687,144.884028,0.000000,113.385096,260.020338,126.133368,288.205755,...,73.111559,166.561962,39.606267,85.204181,146.180017,351.208148,187.838128,461.828117,49.422012,61.847168
7.0,28.152873,157.424646,90.237441,156.811013,98.201755,113.385096,0.000000,370.608933,120.928092,214.210820,...,85.459602,71.554781,99.884947,197.676765,257.790274,261.565200,136.694389,355.179285,67.705016,125.956281
8.0,397.382602,400.807841,369.423165,247.221885,394.416927,260.020338,370.608933,0.000000,360.305082,535.492227,...,299.202006,426.226397,289.362699,187.538534,113.995750,605.119765,432.013832,721.743263,309.422173,285.824563
9.0,138.284448,48.953616,32.936050,113.123738,45.876764,126.133368,120.928092,360.305082,0.000000,175.479655,...,165.354996,107.743673,86.527126,173.205570,249.995729,248.083476,72.374643,379.577216,96.338099,75.810114
10.0,205.321402,141.749208,168.469011,288.308615,143.599814,288.205755,214.210820,535.492227,175.479655,0.000000,...,296.032465,143.446151,251.426990,348.039299,424.051175,78.300250,103.481897,225.479240,243.476995,249.802737


In [22]:
# Read the Excel file
file_path = r'./data/Victims_California_Counties.xlsx'
sheets = pd.ExcelFile(file_path)
sheet_names = sorted(sheets.sheet_names)
county_map = {}
for i, name in enumerate(sheet_names, 1):
    county_map[name] = i
print(county_map)
print(sheet_names)

{'Alameda': 1, 'Alpine': 2, 'Amador': 3, 'Butte': 4, 'Calaveras': 5, 'Colusa': 6, 'Contra Costa': 7, 'Del Norte': 8, 'El Dorado': 9, 'Fresno': 10, 'Glenn': 11, 'Humboldt': 12, 'Imperial': 13, 'Inyo': 14, 'Kern': 15, 'Kings': 16, 'Lake': 17, 'Lassen': 18, 'Los Angeles SPL': 19, 'Madera': 20, 'Marin': 21, 'Mariposa': 22, 'Mendocino': 23, 'Merced': 24, 'Modoc': 25, 'Mono': 26, 'Monterey': 27, 'Napa': 28, 'Nevada': 29, 'Orange': 30, 'Placer': 31, 'Plumas': 32, 'Riverside': 33, 'Sacramento': 34, 'San Benito': 35, 'San Bernardino': 36, 'San Diego': 37, 'San Francisco': 38, 'San Joaquin': 39, 'San Luis Obispo': 40, 'San Mateo': 41, 'Santa Barbara': 42, 'Santa Clara': 43, 'Santa Cruz': 44, 'Shasta': 45, 'Sierra': 46, 'Siskiyou': 47, 'Solano': 48, 'Sonoma': 49, 'Stanislaus': 50, 'Sutter': 51, 'Tehama': 52, 'Trinity': 53, 'Tulare': 54, 'Tuolumne': 55, 'Ventura': 56, 'Yolo': 57, 'Yuba': 58}
['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa', 'Contra Costa', 'Del Norte', 'El Dorado', '

## 1.3 Data Cleaning and Combing

- to create a comprehensive dataset

- Inspect the quantity of the dataset

In [23]:
county_id = 0
all_adj_data = []

for county_id in range(len(sheets.sheet_names)):
    single_county_df = pd.read_excel(file_path, sheet_name=sheet_names[county_id])
    if '2+5' in single_county_df.columns:
        single_county_df.rename(columns={'2+5': '5+2'}, inplace=True)
    if 'Date' in single_county_df.columns:
        single_county_df.rename(columns={'Date': 'Year'}, inplace=True)

    single_county_df = single_county_df[['Year', 1, '5+2', '6+3', '7+4']]
    cols_to_rename = {"Year": 'Date', 1: 'Killed', '5+2': 'Suspected Serious Injury', '6+3': 'Suspected Minor Injury', '7+4': 'Possible Injury'}
    for col in cols_to_rename:
        if col in single_county_df.columns:
            single_county_df.rename(columns={col: cols_to_rename[col]}, inplace=True)
    
    single_county_adj_df = single_county_df.copy()
    single_county_adj_df['County_id'] = county_id + 1
    cols = [c for c in single_county_adj_df.columns if c != 'County_id']
    single_county_adj_df = single_county_adj_df[['County_id'] + cols].iloc[:4383]

    all_adj_data.append(single_county_adj_df.copy())

all_adj_data = pd.concat(all_adj_data, axis=0, ignore_index=True)
print(all_adj_data.shape)

(254214, 6)


In [24]:
all_adj_data.to_csv(r'./all_data_dist.csv', index=False)

In [25]:
import pandas as pd
import numpy as np
all_adj_data = pd.read_csv(r'./all_data_dist.csv')
all_adj_data.head()

,County_id,Date,Killed,Suspected Serious Injury,Suspected Minor Injury,Possible Injury
0,1,2012-01-01 00:00:00,0,1,1,19
1,1,2012-01-02 00:00:00,0,3,4,9
2,1,2012-01-03 00:00:00,0,0,2,18
3,1,2012-01-04 00:00:00,0,0,5,16
4,1,2012-01-05 00:00:00,1,1,8,28


- Integer Validation:
Defines is_integer function to check if values can be converted to integers.
- Filtering and Conversion:
Applies is_integer to filter out non-integer values in columns: 'Killed', 'Suspected Serious Injury', 'Suspected Minor Injury', and 'Possible Injury'.
- Converts these columns to integer type.
- Date Conversion:
Converts 'Date' column to datetime format.
- Data Cleaning:
Drops rows with any missing values, ensuring a clean dataset.

In [26]:
def is_integer(val):
    try:
        return float(val).is_integer()
    except ValueError:
        return False

cols = ['Killed', 'Suspected Serious Injury',
       'Suspected Minor Injury', 'Possible Injury']

for col in cols:
    all_adj_data = all_adj_data[all_adj_data[col].apply(is_integer)]
    all_adj_data.loc[col] = all_adj_data[col].astype(int)

all_adj_data.loc[:, 'Date'] = pd.to_datetime(all_adj_data['Date'], format='mixed')
all_adj_data.dropna(axis=0, inplace=True)
all_adj_data.loc[:, 'Date'] = pd.to_datetime(all_adj_data['Date'], format="mixed").dt.date
all_adj_data['Date'] = all_adj_data['Date'].astype(str)
all_adj_data['County_id'] = all_adj_data['County_id'].astype(int)


### Add Crash Data

In [27]:
file_path = r'./data/California County Crash Victims_ Corrected.xlsx'
sheets = pd.ExcelFile(file_path)
sheet_names = sorted(sheets.sheet_names)
county_id = 0
crash_data = []

for county_id in range(len(sheets.sheet_names)):
    single_county_df = pd.read_excel(file_path, sheet_name=sheet_names[county_id])

    if '1' in single_county_df.columns and 'Fatal (1)' not in single_county_df.columns:
        single_county_df.rename(columns={'1': 'Fatal (1)'}, inplace=True)
    if 1 in single_county_df.columns and 'Fatal (1)' not in single_county_df.columns:
        single_county_df.rename(columns={1: 'Fatal (1)'}, inplace=True)
    if '1.1' in single_county_df.columns:
        single_county_df.rename(columns={'1.1': 'Injury Severe -2'}, inplace=True)
    if '8' in single_county_df.columns:
        single_county_df.rename(columns={'8': 'Injury Other Visible - 3'}, inplace=True)
    if 8 in single_county_df.columns:
        single_county_df.rename(columns={8: 'Injury Other Visible - 3'}, inplace=True)
    if '8.1' in single_county_df.columns:
        single_county_df.rename(columns={'8.1': 'Injury Complaint of Pain - 4'}, inplace=True)

    single_county_df = single_county_df[['Collision Date', 'Fatal (1)', 'Injury Severe -2', 'Injury Other Visible - 3', 'Injury Complaint of Pain - 4']]
    cols_to_rename = {"Collision Date": 'Date', 'Fatal (1)': 'Fatal', 'Injury Severe -2': 'Injury Severe', 
                      'Injury Other Visible - 3': 'Injury Other Visible', 'Injury Complaint of Pain - 4': 'Injury Complaint of Pain'}
    for col in cols_to_rename:
        if col in single_county_df.columns:
            single_county_df.rename(columns={col: cols_to_rename[col]}, inplace=True)

    # Concatenate the original DataFrame and the new list DataFrame horizontally
    single_county_df['County_id'] = county_id + 1
    cols = [c for c in single_county_df.columns if c != 'County_id']
    single_county_df = single_county_df[['County_id'] + cols].iloc[:4383]

    adj = list(adjacency_df.iloc[county_id])
    adj_df = pd.DataFrame([adj], columns=[f'Adjacency_{i+1}' for i in range(len(adj))])

    # Duplicate the row to match the length of the original DataFrame
    adj_df = pd.concat([adj_df] * len(single_county_df), ignore_index=True)

    # Concatenate the original DataFrame and the new list DataFrame horizontally
    if add_adj:
        single_county_adj_df = pd.concat([single_county_df.copy(), adj_df.copy()], axis=1)
    else:
        single_county_adj_df = single_county_df.copy()
    
    crash_data.append(single_county_adj_df.copy())

crash_data = pd.concat(crash_data, axis=0, ignore_index=True)

crash_data.loc[:, 'Date'] = pd.to_datetime(crash_data['Date'], format="mixed").dt.date
crash_data['Date'] = crash_data['Date'].astype(str)
crash_data['County_id'] = crash_data['County_id'].astype(int)
print(crash_data.shape)

(254214, 64)


### Add Population

In [28]:
file_path = r'./data/California Counties Population.xlsx'
sheets = pd.ExcelFile(file_path)
sheet_names = sorted(sheets.sheet_names)

county_map = {'Alameda': 1, 'Alpine': 2, 'Amador': 3, 'Butte': 4, 'Calaveras': 5, 'Colusa': 6, 'Contra Costa': 7, 'Del Norte': 8, 'El Dorado': 9, 'Fresno': 10, 'Glenn': 11, 'Humboldt': 12, 'Imperial': 13, 'Inyo': 14, 'Kern': 15, 'Kings': 16, 'Lake': 17, 'Lassen': 18, 'Los Angeles': 19, 'Madera': 20, 'Marin': 21, 'Mariposa': 22, 'Mendocino': 23, 'Merced': 24, 'Modoc': 25, 'Mono': 26, 'Monterey': 27, 'Napa': 28, 'Nevada': 29, 'Orange': 30, 'Placer': 31, 'Plumas': 32, 'Riverside': 33, 'Sacramento': 34, 'San Benito': 35, 'San Bernardino': 36, 'San Diego': 37, 'San Francisco': 38, 'San Joaquin': 39, 'San Luis Obispo': 40, 'San Mateo': 41, 'Santa Barbara': 42, 'Santa Clara': 43, 'Santa Cruz': 44, 'Shasta': 45, 'Sierra': 46, 'Siskiyou': 47, 'Solano': 48, 'Sonoma': 49, 'Stanislaus': 50, 'Sutter': 51, 'Tehama': 52, 'Trinity': 53, 'Tulare': 54, 'Tuolumne': 55, 'Ventura': 56, 'Yolo': 57, 'Yuba': 58}

all_popu_data = []
pd.set_option("future.no_silent_downcasting", True)

for year in sheet_names:
    single_county_df = pd.read_excel(file_path, sheet_name=year)
    single_county_df.replace(county_map, inplace=True)
    
    date_range = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31')
    
    for county_id in single_county_df.County:
        row = pd.concat([single_county_df.loc[single_county_df['County'] == county_id]] * len(date_range), ignore_index=True)
        row['Date'] = date_range
        all_popu_data.append(row.copy())

all_popu_data = pd.concat(all_popu_data, axis=0, ignore_index=True)
all_popu_data.rename(columns={'County': 'County_id'}, inplace=True)
all_popu_data.loc[:, 'Date'] = pd.to_datetime(all_popu_data['Date'], format='mixed')
all_popu_data['Date'] = all_popu_data['Date'].astype(str)
all_popu_data.shape

(254214, 3)

### Add DVMT

In [29]:
file_path = r'./data/DVMT 2012-2021.xlsx'
sheets = pd.ExcelFile(file_path)
sheet_names = sorted(sheets.sheet_names)

all_dvmt_data = []
pd.set_option("future.no_silent_downcasting", True)

for year in sheet_names:
    single_county_df = pd.read_excel(file_path, sheet_name=year)
    
    date_range = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31')
    
    for county_id in single_county_df.County_ID:
        row = pd.concat([single_county_df.loc[single_county_df['County_ID'] == county_id]] * len(date_range), ignore_index=True)
        row['Date'] = date_range
        all_dvmt_data.append(row.copy())

all_dvmt_data = pd.concat(all_dvmt_data, axis=0, ignore_index=True)
all_dvmt_data.rename(columns={'County_ID': 'County_id'}, inplace=True)
all_dvmt_data.drop(columns=['City_County'], axis=1, inplace=True)
all_dvmt_data.loc[:, 'Date'] = pd.to_datetime(all_dvmt_data['Date'], format='mixed')
all_dvmt_data['Date'] = all_dvmt_data['Date'].astype(str)

all_dvmt_data.shape

(211874, 3)

In [30]:
save_path = ['crash']

if add_adj:
    save_path.append('dist')

if add_vict:
    all_adj_data = pd.merge(crash_data, all_adj_data, on=['County_id', 'Date'], how='outer')
    save_path.append('vict')
else:
    all_adj_data = crash_data.copy()

if add_popu:
    all_adj_data = pd.merge(all_popu_data, all_adj_data, on=['County_id', 'Date'], how='outer')
    save_path.append('popu')

if add_dvmt:
    all_adj_data = pd.merge(all_dvmt_data, all_adj_data, on=['County_id', 'Date'], how='outer')
    save_path.append('dvmt')

all_adj_data.dropna(axis=0, inplace=True)
save_path = './' + '_'.join(save_path) + '.npy'

In [31]:
all_adj_data.to_csv('./dist_vict_popu_dvmt_crash.csv', index=False)

In [32]:
all_adj_data.isna().sum()

County_id                   0
Date                        0
Fatal                       0
Injury Severe               0
Injury Other Visible        0
                           ..
Adjacency_58                0
Killed                      0
Suspected Serious Injury    0
Suspected Minor Injury      0
Possible Injury             0
Length: 68, dtype: int64

In [33]:
print(all_adj_data[all_adj_data.isnull().any(axis=1)])

Empty DataFrame
Columns: [County_id, Date, Fatal, Injury Severe, Injury Other Visible, Injury Complaint of Pain, Adjacency_1, Adjacency_2, Adjacency_3, Adjacency_4, Adjacency_5, Adjacency_6, Adjacency_7, Adjacency_8, Adjacency_9, Adjacency_10, Adjacency_11, Adjacency_12, Adjacency_13, Adjacency_14, Adjacency_15, Adjacency_16, Adjacency_17, Adjacency_18, Adjacency_19, Adjacency_20, Adjacency_21, Adjacency_22, Adjacency_23, Adjacency_24, Adjacency_25, Adjacency_26, Adjacency_27, Adjacency_28, Adjacency_29, Adjacency_30, Adjacency_31, Adjacency_32, Adjacency_33, Adjacency_34, Adjacency_35, Adjacency_36, Adjacency_37, Adjacency_38, Adjacency_39, Adjacency_40, Adjacency_41, Adjacency_42, Adjacency_43, Adjacency_44, Adjacency_45, Adjacency_46, Adjacency_47, Adjacency_48, Adjacency_49, Adjacency_50, Adjacency_51, Adjacency_52, Adjacency_53, Adjacency_54, Adjacency_55, Adjacency_56, Adjacency_57, Adjacency_58, Killed, Suspected Serious Injury, Suspected Minor Injury, Possible Injury]
Index: []

## 1.4 Applying shift window 
- **Function `shift_window(df, W)`**:
  - Sorts DataFrame by `'Date'` and resets index.
  - Prepares target DataFrame `targets_df` from window size `W` onward.
  - Drops `'Date'` column and converts DataFrame to numpy array.
  - Initializes array for windowed data.
  - Fills array with sliding window data.
  - Reshapes and converts windowed data back to DataFrame `features_df`, re-adding expanded dates.
  - Returns `features_df` and `targets_df`.

- **Main Loop**:
  - Iterates through unique `County_id` values in `all_adj_data`.
  - Applies `shift_window` to each county's data with window size `W = 7`.
  - Appends results to `all_features_df` and `all_targets_df`.

This process prepares windowed features and targets for each county, facilitating time series analysis.


In [34]:
def shift_window(df, W):
    # cols = ['Date', 'Killed', 'Suspected Serious Injury',
    #         'Suspected Minor Injury', 'Possible Injury']
    cols = ['Date', 'Fatal', 'Injury Severe', 
            'Injury Other Visible', 'Injury Complaint of Pain']
    df_sorted = df.sort_values(by='Date')
    df = df_sorted.reset_index(drop=True)
    date = df[['Date']]
    
    data = df.values
    M, N = df.shape
    targets = data[W:M, :]
    targets_df = pd.DataFrame(targets, index=range(W, M), columns=df.columns)
    targets_df = targets_df[cols]
    targets_df['Date'] = pd.to_datetime(targets_df['Date'], format='mixed')
    # Step 1: Convert DataFrame to numpy array

    df = df.drop(columns=['Date'], axis=1)
    data = df.values
    M, N = df.shape

    # Step 2: Create a new array to store the windows
    result = np.empty((M - W, W, N))

    # Step 3: Fill the result array with windowed data
    for i in range(M - W):
        result[i] = data[i:i+W]

    # Convert features to DataFrame
    features_flat = result.reshape(-1, N)
    index_features = pd.MultiIndex.from_product([range(M - W), range(W)], names=['row', 'window'])
    features_df = pd.DataFrame(features_flat, index=index_features, columns=df.columns)

    dates_expanded = np.array([date[i:i+W] for i in range(M - W)]).reshape(-1)
    features_df['Date'] = pd.to_datetime(dates_expanded, format='mixed')

    return features_df, targets_df

window_size = 7
all_features_df = []
all_targets_df = []
for county_id in range(1, len(all_adj_data['County_id'].unique()) + 1):
    features_df, targets_df = shift_window(all_adj_data.loc[all_adj_data['County_id'] == county_id], window_size)
    all_features_df.append(features_df.copy())
    all_targets_df.append(targets_df.copy())


## 1.5 Data Split

- Initialize subsets

- filtering the subset contain the last month of the last year

In [35]:
from datetime import timedelta

X_train_df, y_train_df, X_test_df, y_test_df = [], [], [], []
for county_id in range(1, len(all_adj_data['County_id'].unique()) + 1):
    single_x = all_features_df[county_id-1]
    single_y = all_targets_df[county_id-1]
    # print(len(single_x), len(single_y))

    latest_date = single_x['Date'].max()

    # Calculate the start date for the one-month range
    one_month_ago = latest_date - timedelta(days=30)

    # Filter the DataFrame to get the last one month of data
    i, j = single_x[single_x['Date'] == one_month_ago].index[0]
    single_x['year'] = single_x['Date'].dt.year
    single_x['month'] = single_x['Date'].dt.month
    single_x['day_of_week'] = single_x['Date'].dt.dayofweek
    single_x.drop(columns=['Date'], axis=1, inplace=True)
    test_data = single_x.loc[(i+1, 0): ]
    train_data = single_x.loc[:(i, j)]
    
    last_row = single_y[single_y['Date'] > one_month_ago].index[0]
    single_y.drop(columns=['Date'], axis=1, inplace=True)
    test_label = single_y.loc[last_row+1:]
    train_label = single_y.loc[:last_row]
    train_label = train_label.apply(pd.to_numeric)
    test_label = test_label.apply(pd.to_numeric)
    
    X_train_df.append(train_data.copy())
    X_test_df.append(test_data.copy())
    y_train_df.append(train_label.copy())
    y_test_df.append(test_label.copy())


- **Function Definition**:
  - `pandas_to_numpy(df)`: Converts a multi-index DataFrame to a 3D numpy array.

- **Combining Data**:
  - Vertically stacks the arrays in `X_train`, `y_train`, `X_test`, and `y_test` to form final 3D arrays.

This process prepares and consolidates training and testing datasets into 3D numpy arrays for machine learning models.


In [36]:
def pandas_to_numpy(df):
    M = len(df.index.get_level_values('row').unique())
    W = len(df.index.get_level_values('window').unique())
    N = df.shape[1]
    array_3d = df.values.reshape(M, W, N)
    return array_3d

X_train, y_train, X_test, y_test = [], [], [], []
for i in range(len(X_train_df)):
    x_obj = pandas_to_numpy(X_train_df[i])
    y_obj = y_train_df[i].values
    X_train.append(x_obj)
    y_train.append(y_obj)

for i in range(len(X_test_df)):
    x_obj = pandas_to_numpy(X_test_df[i])
    y_obj = y_test_df[i].values
    X_test.append(x_obj)
    y_test.append(y_obj)

X_train = np.vstack(X_train)
y_train = np.vstack(y_train)
X_test = np.vstack(X_test)
y_test = np.vstack(y_test)

In [37]:
print(save_path)
with open(save_path, 'wb') as f:
    np.save(f, np.array([X_train, y_train, X_test, y_test], dtype='object'))

./crash_dist_vict.npy


In [38]:
X_train.shape

(252066, 7, 70)

This indicates that the X_train dataset is a 3-dimensional numpy array with the following dimensions:

- 252068: This is the number of samples in training dataset.
- 7: This represents the window size, meaning each sample consists of 7 time steps (or windows).
- 66: This is the number of features for each time step in the window.